In [1]:
import sys
import os
from os.path import isfile, join, split
import glob
import pandas as pd

import ast

In [2]:
files = pd.DataFrame([file for file in glob.glob("param_tests/*.csv")], columns=["fullpath"])
files_split = files['fullpath'].str.rsplit(os.sep, 1, expand=True).rename(columns={0: 'path', 1: 'filename'})
files = files.join(files_split)
files.head()



,fullpath,path,filename
0,param_tests/Burger_etal_tpe_BiTRRT.csv,param_tests,Burger_etal_tpe_BiTRRT.csv
1,param_tests/Cano_etal_tpe_BKPIECE.csv,param_tests,Cano_etal_tpe_BKPIECE.csv
2,param_tests/Burger_etal_tpe_BKPIECE.csv,param_tests,Burger_etal_tpe_BKPIECE.csv
3,param_tests/Cano_etal_gp_BKPIECE.csv,param_tests,Cano_etal_gp_BKPIECE.csv
4,param_tests/Cano_etal_rand_RRTConnect.csv,param_tests,Cano_etal_rand_RRTConnect.csv


In [48]:
RESULTS_DIR = '../output/final'
PLANNER_SELECT = 'Cano_etal'
PLANNER = 'BKPIECE'
MODE = 'rand'


fps = [y for x in os.walk(RESULTS_DIR)
                    for y in glob.glob(join(x[0], '*.csv'))
                        if (PLANNER_SELECT in y) and 
                            (PLANNER in y) and
                            (MODE in y) ]

# Returns a dict of the best run out of all runs with the min loss
best_run = {'run': 0, 'min_loss': float('inf')}
for fp in fps:
    df = pd.read_csv(fp, index_col=False)
    df_best_run = df.iloc[df['loss'].idxmin(axis=1)]

    if df_best_run['loss'] < best_run['min_loss']:
        best_run['min_loss'] = df_best_run['loss']
        best_run['min_run'] = df_best_run.to_dict()
        best_run['run'] = int(fp.split(os.sep)[-2].split('_')[-1])

# Extracts the params set str as dict and sets planner params
params_set = ast.literal_eval(best_run['min_run']['params'])

params_set

{'border_fraction': 0.35000000000000003,
 'projection_evaluator': 'joints(crane_plus_shoulder_revolute_joint, crane_plus_wrist_joint)',
 'range': 0.8,
 'failed_expansion_score_factor': 0.8,
 'longest_valid_segment_fraction': 0.005,
 'min_valid_path_fraction': 0.1,
 'type': 'geometric::BKPIECE'}

In [65]:
default_fps = [y for x in os.walk(RESULTS_DIR)
                            for y in glob.glob(join(x[0], '*.csv'))
                                if ('default' in y) and
                                (PLANNER_SELECT in y)]
default_df = pd.DataFrame()
for f in default_fps:
    df = pd.read_csv(f, index_col=False, sep=',').dropna(axis=1)
    df = df[df['planner'].str.contains(PLANNER)]
    default_df = pd.concat([default_df, df])
default_df

avg_default = {'run': 'avg_default', 'avg_runs': default_df['avg_runs'].mean(), 
               't_avg_run_time': default_df['t_avg_run_time'].mean(),
               't_avg_plan_time': default_df['t_avg_plan_time'].mean(),
               't_avg_dist': default_df['t_avg_dist'].mean(), 
               't_avg_path_length': default_df['t_avg_path_length'].mean(),
               't_avg_success': default_df['t_avg_success'].mean()}

In [70]:
df_avg_default= pd.DataFrame(avg_default, index=[0])
df_avg_default

,run,avg_runs,t_avg_run_time,t_avg_plan_time,t_avg_dist,t_avg_path_length,t_avg_success
0,avg_default,5.0,1.481679,1.005983,1.58939,7.525043,0.933333


In [79]:
default_df.iloc[0]['params']

"{'longest_valid_segment_fraction': 0.005, 'border_fraction': 0.5, 'projection_evaluator': 'joints(crane_plus_shoulder_revolute_joint, crane_plus_wrist_joint)', 'range': 1.0, 'failed_expansion_score_factor': 0.5, 'min_valid_path_fraction': 0.5, 'type': 'geometric::BKPIECE'}"

In [37]:
params_set = ast.literal_eval(min['min_run']['params'])
params_set.update({'type':'geometric::'+PLANNER})

params_set

{'border_fraction': 0.35000000000000003,
 'projection_evaluator': 'joints(crane_plus_shoulder_revolute_joint, crane_plus_wrist_joint)',
 'range': 0.8,
 'failed_expansion_score_factor': 0.8,
 'longest_valid_segment_fraction': 0.005,
 'min_valid_path_fraction': 0.1,
 'type': 'geometric::BKPIECE'}

In [29]:
dfs[0]['loss'].idxmin(axis=1)

NameError: name 'dfs' is not defined

In [ ]:
min = dfs[0].iloc[dfs[0]['loss'].idxmin(axis=1)]

In [ ]:
min['loss']

In [ ]:
min['f'] = 2